In [2]:
# 批量标准化
# 为了应对深度模型训练的挑战，在训练模型时，批量归一化利用小批量的
# 上的均值和标准差，不断调整神经网络中间输出，从而使整个神经网络在各层的
# 的中间输出的数值更加稳定。
# 批量归一化和残差网络为训练和设计深度模型提供了两类重要的思路

# 批量归一化层
# 对全连接层和卷积层做批量归一化的方法稍有不同

# 对全连接层的批量归一化
# 仿射变换之后，激活函数之前，f（BN(x)）,X_hat = BN(x)引入了两个参数，拉伸参数gamma和偏移参数beta
# 这两个参数和X_hat的形状相同,按照元素乘法和加法运算，这两个参数是可学习的，并且按照理论，
# 如果学出来的gamma和beta满足最终的y与x相同，则表示学出的模型没有批量归一化的必要，符合逻辑

# 对卷积层的批量归一化
# 在卷积计算之后，应用激活函数之前，如果卷及计算输出多个通道，需要对这些通道的输出分别做批量归一化
# 每个通道都有独立的拉伸和偏移参数，均为标量。

# 预测时的批量归一化
# 从零开始实现

import time
import torch
from torch import nn,optim
import torch.nn.functional as F

import sys
sys.path.append("..")
import d2lzh_pytorch as d2l
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def batch_norm(is_training,x,gamma,beta,moving_mean,moving_var,eps,momentum):
    if not is_training:
        # 如果是在预测模式下，直接使用传入的移动平均所得到的均值和方差
        x_hat = (x - moving_mean)/torch.sqrt(moving_var + eps)
    else:
        assert len(x.shape) in (2,4)
        if len(x.shape) == 2:
            # 使用全连接层的情况，计算特征维上的均值和方差
            mean = x.mean(dim=0)
            var = ((x-mean)**2).mean(dim=0)
        else:
            # 使用二维卷积层的情况，计算通道维上（axis=1）的均值和方差，这里我们需要保持
            # x的形状以便后面做广播运算
            mean = x.mean(dim=0,keepdim=True).mean(dim=2,keepdim=True).mean(dim=3,keepdim=True)
            var = ((x - mean)**2).mean(dim=0,keepdim=True).mean(dim=2,keepdim=True).mean(dim=3,keepdim=True)
        # 训练模式下使用当前的均值和方差做标准化
        x_hat = (x-mean)/torch.sqrt(var+eps)
        # 更新移动平均的均值和方差
        moving_mean = momentum*moving_mean + (1.0 - momentum)*mean
        moving_var = momentum*moving_var + (1.0 - momentum)*var
    Y = gamma * x_hat + beta # 拉伸和偏移
    return Y,moving_mean,moving_var



In [ ]:
# 接下来自定义一个BatchNorm层

class BatchNorm(nn.Module):
    def __init__(self,num_features,num_dims):
        super(BatchNorm,self).__init__()
        if num_dims == 2:
            shape = (1,num_features)
        else:
            shape = (1,num_features,1,1)
        # 参与求梯度和迭代的拉伸和偏移参数，分别初始化成0和1
        self.gamma = nn.Parameter(torch.ones(shape))
        self.beta = nn.Parameter(torch.zeros(shape))
        # 不参与求梯度和迭代的变量，全在内存上初始化成0
        self.moving_mean = torch.zeros(shape)
        self.moving_var = torch.zeros(shape)
        
    def forward(self,x):
        # 如果x不在内存上，将moving_mean和moving_var复制到x所在的显存上
        if self.moving_mean.device != x.device:
            self.moving_mean = self.moving_mean.to(x.device)
            self.moving_var = self.moving_var.to(x.device)
        Y,self.moving_mean,self.moving_var = batch_norm(self.traing,x,self.gamma,self.beta,self.moving_mean,
                                                       self.moving_var,eps = 1e-5,momentum=0.9)
        return Y
            

In [ ]:
# 使用批量归一化层的LeNet
net = nn.Sequential(
    nn.Conv2d(1,6,5),
    BatchNorm(6,num_dims = 4),
    nn.Sigmoid(),
    nn.MaxPool2d(2,2), # kernel_size,stride
    
    nn.Conv2d(6,16,5),
    BatchNorm(16,num_dims=4),
    nn.Sigmoid(),
    nn.MaxPool2d(2,2),
    
    d2l.FalttenLayer(),
    nn.Linear(16*4*4,120),
    BatchNorm(120,num_dims=2),
    nn.Sigmoid(),
    
    nn.Linear(120,84),
    BatchNorm(84,num_dims=2),
    nn.Sigmoid(),
    
    nn.Linear(84,10)
    
)

In [ ]:
# 简洁实现   直接使用nn库中的BatchNorm1d和BatchNorm2d来实现（1d是全连接层，2d是卷积层的BN）
net = nn.Sequential(
            nn.Conv2d(1, 6, 5), # in_channels, out_channels, kernel_size
            nn.BatchNorm2d(6),
            nn.Sigmoid(),
            nn.MaxPool2d(2, 2), # kernel_size, stride
            nn.Conv2d(6, 16, 5),
            nn.BatchNorm2d(16),
            nn.Sigmoid(),
            nn.MaxPool2d(2, 2),
            d2l.FlattenLayer(),
            nn.Linear(16*4*4, 120),
            nn.BatchNorm1d(120),
            nn.Sigmoid(),
            nn.Linear(120, 84),
            nn.BatchNorm1d(84),
            nn.Sigmoid(),
            nn.Linear(84, 10)
        )